In [ ]:
my_ProjectID = "medical-imaging-ai"

In [ ]:
import os

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
# CT image sample
#my_SeriesInstanceUID = "1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095"
my_StudyInstanceUID = "1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095"
collection_ID = "lctsc"
# pathology image sample
#my_SeriesInstanceUID = "1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0"

In [ ]:
%%bigquery idc_summary_df --project=$my_ProjectID 

SELECT
  COUNT(DISTINCT(PatientID)) AS total_patients,
  COUNT(DISTINCT(StudyInstanceUID)) AS total_studies,
  COUNT(DISTINCT(SeriesInstanceUID)) AS total_series,
  COUNT(DISTINCT(SOPInstanceUID)) AS total_instances,
  COUNT(DISTINCT(collection_id)) AS total_collections,
  SUM(instance_size)/(POW(1024,4)) AS total_size_TB
FROM
  `bigquery-public-data.idc_current.dicom_all`

In [ ]:
idc_summary_df

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(my_ProjectID)

selection_query = f"\
  SELECT  \
    StudyInstanceUID, \
    SeriesInstanceUID, \
    SOPInstanceUID, \
    gcs_url \
  FROM \
    `bigquery-public-data.idc_current.dicom_all` \
  WHERE \
    StudyInstanceUID = \"{my_StudyInstanceUID}\""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(my_ProjectID)

selection_query = f"\
  WITH studies AS (SELECT DISTINCT StudyInstanceUID FROM `bigquery-public-data.idc_current.dicom_all` \
                  WHERE collection_id =  \"{collection_ID}\"  \
                    LIMIT 10) \
  SELECT  \
    StudyInstanceUID, \
    SeriesInstanceUID, \
    SOPInstanceUID, \
    gcs_url \
  FROM \
    `bigquery-public-data.idc_current.dicom_all` INNER JOIN studies USING(StudyInstanceUID) \
    "

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [ ]:
collection_ID

In [ ]:
selection_query = f"\
  SELECT DISTINCT StudyInstanceUID FROM `bigquery-public-data.idc_current.dicom_all` \
        WHERE collection_id =  \"{collection_ID}\"  \
         LIMIT 10"

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [ ]:
pd.set_option('display.max_colwidth', None)

selection_df

In [ ]:
# save the list of files
# save the list of GCS URLs into a file
import os
idc_download_folder = "./IDC_downloads"
if not os.path.exists(idc_download_folder):
  os.mkdir(idc_download_folder)

selection_manifest = os.path.join(idc_download_folder, "lctsc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [ ]:
# copy files to staging area
# download is this simple!
!cat ./IDC_downloads/lctsc_manifest.txt | gsutil -m cp -I gs://miai-staging_bucket/LCTSC

In [ ]:
!gsutil ls gs://miai-staging_bucket/CPTAC-LSCC